### Библиотеки / данные

импортируем numpy и pandas

In [2]:
import numpy as np
import pandas as pd

задаем некоторые настройки pandas, регулирующие формат вывода

In [ ]:
pd.options.display.max_rows = 10

- считываем данные 
- используем столбец Symbol в качестве индекса 
- считываем только те столбцы ['Symbol', 'Sector', 'Price', 'Book Value']

| Column Name        | Description
| ------------- |:-------------:|
|Symbol|Сокращенное название организации|
|Name|Полное название организации|
|Sector|Сектор экономики|
|Price|Стоимость акции|
|Dividend Yield|Дивидендная доходность|
|Price/Earnings|Цена / прибыль|
|Earnings/Share|Прибыль на акцию|
|Book Value|Балансовая стоимость компании|
|52 week low|52-недельный минимум|
|52 week high|52-недельный максимум|
|Market Cap|Рыночная капитализация|
|EBITDA|**E**arnings **b**efore **i**nterest, **t**axes, **d**epreciation and **a**mortization|
|Price/Sales|Цена / объём продаж|
|Price/Book|Цена / балансовая стоимость|
|SEC Filings|Ссылка *sec.gov*|

In [16]:
sp500 = pd.read_csv("../data/sp500.csv",
                    index_col='Symbol',
                    usecols=['Symbol', 'Sector', 'Price', 'Book Value'])

In [ ]:
sp500.head()

считываем исторические данные о котировках акций

In [26]:
omh = pd.read_csv('../data/omh.csv', 
                  parse_dates=['Date'])

omh.set_index('Date', 
              inplace=True)

In [ ]:
omh.head()

### Сводка статистик

получаем сводку статистик для датафрейма, с которой работаем как с обычным датафреймом

In [ ]:
sp500.describe()

вычисляем сводку статистик для отдельного столбца Price

In [ ]:
sp500.Price.describe()

получаем сводку статистик для нечисловых данных

In [ ]:
sp500.Sector.describe()

метод info:

In [ ]:
sp500.info()

получаем сводную статистику для нечисловых данных

In [ ]:
sp500.Sector.value_counts(normalize=True)

### Арифметические операции

- задаем стартовое значение генератора случайных чисел для получения воспроизводимых результатов
- создаем объект DataFrame

In [ ]:
np.random.seed(123)
df = pd.DataFrame(np.random.randn(5, 4), 
                  columns=['A', 'B', 'C', 'D'])
df

умножаем все на 2, берём только абсолютные значения

In [ ]:
abs(df * 2)

вычитаем первую строку из каждой строки объекта DataFrame

In [ ]:
df

In [ ]:
df.iloc[0]

In [ ]:
df - df.iloc[0]

вычитаем объект DataFrame из объекта Series

In [ ]:
df.iloc[0] - df

- возьмём второе и третье поле 1-ой строки:
- добавляем столбец E
- смотрим, как применяется выравнивание в этой математической операции

In [ ]:
df

In [ ]:
s = df.iloc[0][1:3]
s['E'] = 0
s

In [ ]:
df + s

извлекаем строки в позициях с 1-й по 3-ю и только столбцы B и C <br>
по сути - извлекаем небольшой квадрат из середины df

In [ ]:
subframe = df[1:4][['B', 'C']].copy()
subframe

демонстрируем, как происходит выравнивание при выполнении операции вычитания

In [ ]:
df - subframe

извлекаем столбец A и вычитаем его из нашего датафрейма

In [ ]:
df.sub(df['A'], axis=0)

### Одномерные статистики

#### минимум / максимум

определяем максимальную цену для обеих акций

In [ ]:
omh[['MSFT', 'AAPL']].min()

определяем индекс, которому соответствует максимальная цена для обеих акций

In [ ]:
omh[['MSFT', 'AAPL']].idxmin()

#### cреднее значение / медиана / мода

<img src='..\images\moda-mediana.jpg'/>

вычисляем среднее значение для всех столбцов в датафрейме omh

In [ ]:
omh.mean()

вычисляем значение, усредненное по всем столбцам, для каждой строки (выведем первые 5)

In [ ]:
omh.mean(axis=1).head() 

вычисляем медиану значений для каждого столбца

In [ ]:
omh.median()

вычисляем моду для столбца Sector

In [ ]:
sp500.Sector.mode()

мод может быть несколько, поэтому результат операции - Series 

In [ ]:
s = pd.Series([1, 2, 3, 3, 5, 1])
s.mode()

#### [дисперсия](https://ru.wikipedia.org/wiki/Дисперсия_случайной_величины) / среднеквадратичное отклонение

вычисляем дисперсию значений в каждом столбце

In [ ]:
omh.var()

In [ ]:
(omh.MSFT**2 - omh.MSFT.mean()**2).sum() / (omh.shape[0]-1)

вычисляем среднеквадратичное отклонение

In [ ]:
omh.std()

In [ ]:
omh.MSFT.var()**0.5

#### [ковариация](https://ru.wikipedia.org/wiki/Ковариация) / [корреляция](https://ru.wikipedia.org/wiki/Корреляция)

вычисляем ковариацию между MSFT и AAPL

In [ ]:
omh.MSFT.cov(omh.AAPL)

вычисляем корреляцию между MSFT и AAPL

In [ ]:
omh.MSFT.corr(omh.AAPL)

In [ ]:
omh.MSFT.cov(omh.AAPL) / (omh.MSFT.std() * omh.AAPL.std())

либо можем получать матрицу ковариаций

In [ ]:
omh.corr()

### Преобразование данных 

#### дискретизация и квантилизация

генерируем 10000 случайных чисел из стандартного нормального распределения

In [3]:
np.random.seed(123456)
dist = np.random.normal(size = 10000)
dist

array([ 0.4691123 , -0.28286334, -1.5090585 , ...,  0.26296448,
       -0.83377412, -0.10418135])

выводим среднее и стандартное отклонение

In [4]:
(dist.mean(), dist.std())

(-0.002863324040906651, 1.008716203199891)

In [5]:
pd.cut(dist, 5)

[(-0.633, 0.81], (-0.633, 0.81], (-2.077, -0.633], (-2.077, -0.633], (0.81, 2.254], ..., (-2.077, -0.633], (-0.633, 0.81], (-0.633, 0.81], (-2.077, -0.633], (-0.633, 0.81]]
Length: 10000
Categories (5, interval[float64, right]): [(-3.528, -2.077] < (-2.077, -0.633] < (-0.633, 0.81] < (0.81, 2.254] < (2.254, 3.698]]

разбиваем на пять одинаковых по размеру групп (по размеру интервалов - не количеству наблюдений в группе!)

In [7]:
bins = pd.cut(dist, 5)
bins

[(-0.633, 0.81], (-0.633, 0.81], (-2.077, -0.633], (-2.077, -0.633], (0.81, 2.254], ..., (-2.077, -0.633], (-0.633, 0.81], (-0.633, 0.81], (-2.077, -0.633], (-0.633, 0.81]]
Length: 10000
Categories (5, interval[float64, right]): [(-3.528, -2.077] < (-2.077, -0.633] < (-0.633, 0.81] < (0.81, 2.254] < (2.254, 3.698]]

найдём длины соответствующих интервалов

In [9]:
bins.categories

IntervalIndex([(-3.528, -2.077], (-2.077, -0.633], (-0.633, 0.81], (0.81, 2.254], (2.254, 3.698]], dtype='interval[float64, right]')

In [10]:
[q.right - q.left for q in bins.categories]

[1.451, 1.444, 1.443, 1.444, 1.444]

генерируем 50 значений возраста в диапазоне от 6 до 70

In [11]:
np.random.seed(242)
ages = np.random.randint(6, 70, 50)
ages

array([20, 45,  8, 28, 53,  7, 12, 19, 66, 42, 10, 37, 34, 25, 44, 69, 34,
       67, 25, 51,  8, 50, 33, 42, 20, 31, 64, 51, 59, 11, 40, 32, 33, 29,
       28, 18, 16, 47, 45, 66, 43, 65, 16, 17, 69,  9, 19, 42, 33, 39])

добавляем имена для групп

In [12]:
ranges = [6, 12, 18, 35, 50, 70]
labels = ['Youth', 'Young Adult', 'Adult', 'Middle Aged', 'Retired persons']
agebins = pd.cut(ages, ranges, labels=labels)
agebins.describe()

,counts,freqs
categories,,
Youth,7,0.14
Young Adult,4,0.08
Adult,16,0.32
Middle Aged,12,0.24
Retired persons,11,0.22


разбиваем (используя квантили) на 5 групп с одинаковым количеством элементов

In [13]:
qbin = pd.qcut(dist, 5)

найдём статистику по полученным группам

In [14]:
qbin.describe()

,counts,freqs
categories,,
"(-3.522, -0.861]",2000,0.2
"(-0.861, -0.241]",2000,0.2
"(-0.241, 0.261]",2000,0.2
"(0.261, 0.866]",2000,0.2
"(0.866, 3.698]",2000,0.2


пример использования qcut:

In [17]:
sp500_copy = sp500.copy()
sp500_copy['Price_Group'], bins = pd.qcut(sp500_copy.Price, 
                                          5,
                                          labels=['group_'+str(i) for i in range(1, 6)],
                                          retbins=True)
sp500_copy.Price_Group

Symbol
MMM     group_5
ABT     group_2
ABBV    group_3
ACN     group_4
ACE     group_5
         ...   
YHOO    group_2
YUM     group_4
ZMH     group_5
ZION    group_1
ZTS     group_1
Name: Price_Group, Length: 500, dtype: category
Categories (5, object): ['group_1' < 'group_2' < 'group_3' < 'group_4' < 'group_5']

In [18]:
bins

array([   0.   ,   34.558,   50.732,   69.586,   97.032, 1197.12 ])

In [19]:
sp500_copy.Price_Group.value_counts()

group_1    100
group_2    100
group_3    100
group_4    100
group_5    100
Name: Price_Group, dtype: int64

#### кумулятивные суммы

вычисляем кумулятивную сумму

In [21]:
pd.Series([1, 2, 3, 4])

0    1
1    2
2    3
3    4
dtype: int64

In [20]:
pd.Series([1, 2, 3, 4]).cumsum()

0     1
1     3
2     6
3    10
dtype: int64

вычисляем кумулятивное произведение

In [22]:
pd.Series([1, 2, 3, 4]).cumprod()

0     1
1     2
2     6
3    24
dtype: int64

#### ранжирование

для примера:

In [23]:
s = pd.Series([160, 165, 165, 170, 175], index=list('abcde'))
s

a    160
b    165
c    165
d    170
e    175
dtype: int64

ранжируем значения

In [24]:
s.rank()

a    1.0
b    2.5
c    2.5
d    4.0
e    5.0
dtype: float64

#### относительное изменение

In [27]:
omh[['MSFT']].head()

,MSFT
Date,
2014-12-01,48.62
2014-12-02,48.46
2014-12-03,48.08
2014-12-04,48.84
2014-12-05,48.42


вычисляем относительнон изменение для MSFT (текущее значение с предыдущим)

In [ ]:
omh[['MSFT']].pct_change().head()

In [ ]:
(48.46 - 48.62)/48.62

### Оконные функции

Объект Rolling:

In [28]:
r = omh.MSFT.rolling(3)

возможные операции:

In [33]:
#r.max()

скользящее среднее:

In [34]:
r.mean()

Date
2014-12-01          NaN
2014-12-02          NaN
2014-12-03    48.386667
2014-12-04    48.460000
2014-12-05    48.446667
2014-12-08    48.320000
2014-12-09    47.903333
2014-12-10    47.396667
2014-12-11    47.220000
2014-12-12    47.006667
2014-12-15    46.930000
2014-12-16    46.260000
2014-12-17    45.856667
2014-12-18    46.140000
2014-12-19    46.973333
2014-12-22    47.720000
2014-12-23    48.030000
2014-12-24    48.190000
2014-12-26    48.156667
2014-12-29    47.823333
2014-12-30    47.450000
2014-12-31    46.973333
Name: MSFT, dtype: float64

первое значение:

In [ ]:
omh.MSFT.loc['2014-12-01':'2014-12-03'].mean()

второе:

In [ ]:
omh.MSFT.loc['2014-12-02':'2014-12-04'].mean()